<a href="https://colab.research.google.com/github/NadezhdaMalysheva/projects/blob/main/Experiments_with_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Подгружаем необходимые библиотеки 

In [ ]:
pip install pytorch-tabnet

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.offline as py
color = sns.color_palette()
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.tools as tls

%matplotlib inline

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import joblib
from tqdm.notebook import tqdm

from pytorch_tabnet.tab_model import TabNetClassifier

from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

import lightgbm as lgb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

from time import time
import joblib

In [ ]:
import torch

#Загружаем данные, параметры

In [ ]:
target = 'class'

add_columns = ['subClass', 'objID', 'z', 'zErr', 'ra', 'dec']

photo_columns = ['psfMag_u',	'psfMag_g',	'psfMag_r',	'psfMag_i',	'psfMag_z',
                 'cModelMag_u',	'cModelMag_g',	'cModelMag_r',	'cModelMag_i',	'cModelMag_z']

feature_columns = (
    photo_columns + add_columns + [target])

In [ ]:
df = joblib.load('/content/drive/MyDrive/Научная работа/Спецсем/TabNetModel/df_agg.pkl')#pd.read_csv('/content/drive/MyDrive/Научная работа/Спецсем/TabNetModel/df_agg.csv')
agr_feature = [x for x in df.columns if x not in feature_columns]

In [ ]:
def data_preparation(X, y, c=10000, test_size = 0.8):

    X1_train, X1_test, y1_train, y1_test = train_test_split(X[y==1], y[y==1], test_size=test_size, random_state = 43)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X[y==2], y[y==2], test_size=test_size, random_state = 43)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X[y==3], y[y==3], test_size=test_size, random_state = 43)
    
    count = c
    count1 = c

    X_train, X_test = np.concatenate((X1_train[:count], X2_train[:count], X3_train[:count])), np.concatenate((X1_test[:count1], X2_test[:count1], X3_test[:count1]))
    y_train, y_test = np.concatenate((y1_train[:count], y2_train[:count], y3_train[:count])), np.concatenate((y1_test[:count1], y2_test[:count1], y3_test[:count1]))
    

    return [X_train, X_test, y_train, y_test, X1_train, X1_test, y1_train, y1_test, X2_train, X2_test, y2_train, y2_test, X3_train, X3_test, y3_train, y3_test]

In [ ]:
def scor(y_test, y_pred):
  return accuracy_score(y_test, y_pred)

lgb_reg_params = {
    'learning_rate':    hp.uniform('learning_rate', 0.001, 0.1),
    'min_child_samples':hp.randint('min_child_samples', 50)+1,
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 0.9),
    'num_leaves' :      hp.randint('num_leaves', 100)+10,
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.99),
    'n_estimators':     hp.randint('n_estimators', 1500)+100
}
lgb_fit_params = {
    'early_stopping_rounds': 20,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['score'] = lambda y, pred: -accuracy_score(y, pred)


rf_reg_params = {
    'min_samples_leaf': hp.randint('min_samples_leaf', 20)+1,
    'min_samples_split':hp.uniform('min_samples_split', 0.001, 0.1),
    #'max_features':     hp.choice('max_features', ['auto', 'sqrt', 'log2', None]),
    #'learning_rate':    hp.uniform('learning_rate', 0.001, 0.1),
    'n_estimators':     hp.randint('n_estimators', 800)+100
}
rf_fit_params = {
}
rf_para = dict()
rf_para['reg_params'] = rf_reg_params
rf_para['fit_params'] = rf_fit_params
rf_para['score'] = lambda y, pred: -accuracy_score(y, pred)

tabnet_reg_params = {
    'n_d' :              64,
    'n_a' :              64,
    'n_steps' :          hp.randint('n_steps', 10-3)+3,
    'gamma' :            hp.uniform('gamma', 1.0, 3.0),
    'lambda_sparse' :    hp.uniform('lambda_sparse', 0.0, 0.01),
    'momentum' :         0.3, 
    'clip_value' :       2.,
    'optimizer_params' : dict(lr=2e-2),
    'scheduler_params' : {"step_size":50, "gamma":0.9},
    'scheduler_fn' :     torch.optim.lr_scheduler.StepLR,
    'mask_type' :       'entmax'
}

tabnet_fit_params = {
    'max_epochs' : 100, 
    'patience' : 15,
    'batch_size' : 512,
    'virtual_batch_size' : 128,
    'num_workers' : 0,
    'weights' : 1,
    'drop_last' : False,
    #'from_unsupervised' : unsupervised_model
}
tabnet_para = dict()
tabnet_para['reg_params'] = tabnet_reg_params
tabnet_para['fit_params'] = tabnet_fit_params
tabnet_para['score'] = lambda y, pred: -accuracy_score(y, pred)

In [ ]:
target = 'class'

add_columns = ['subClass', 'objID', 'z', 'zErr', 'ra', 'dec']

photo_columns = ['psfMag_u',	'psfMag_g',	'psfMag_r',	'psfMag_i',	'psfMag_z',
                 'cModelMag_u',	'cModelMag_g',	'cModelMag_r',	'cModelMag_i',	'cModelMag_z']

feature_columns = (
    photo_columns + add_columns + [target])

In [ ]:
X, y  = joblib.load('/content/drive/MyDrive/Научная работа/Спецсем/TabNetModel/X_y.pkl')

# Общая функция

In [ ]:
with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'w') as f:
  pass

In [ ]:
def bootstrap_accuracy(model, X_test, y_test):
  from sklearn.utils import resample
  from matplotlib import pyplot
  values = np.concatenate((X_test, y_test.reshape((len(y_test), 1))), axis=1)
  n_iterations = 100
  n_size = int(len(y_test) * 0.50)
  stats = list()
  for i in range(n_iterations):
    test = resample(values, n_samples=n_size)
    predictions = model.predict(test[:,:-1])
    score = accuracy_score(test[:,-1], predictions)
    stats.append(score)
  #pyplot.hist(stats, range=(0.872, 0.9))
  #pyplot.show()
  alpha = 0.97
  p = ((1.0-alpha)/2.0) * 100
  lower = max(0.0, np.percentile(stats, p))
  p = (alpha+((1.0-alpha)/2.0)) * 100
  upper = min(1.0, np.percentile(stats, p))
  main = np.mean(stats)
  #print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))
  return main, np.max([main-lower, upper-main])

In [ ]:
def bootstrap(X_test, y_test, X_valid, y_valid):
  from sklearn.utils import resample
  from matplotlib import pyplot
  values = np.concatenate((X_test, y_test.reshape((len(y_test), 1))), axis=1)
  n_iterations = 100
  n_size = int(len(y_test) * 0.50)
  stats = list()
  for i in range(n_iterations):
    train_ind = np.random.randint(0, len(values), n_size)
    test_ind = np.setdiff1d(range(len(values)), train_ind)
    train = values[train_ind]
    test = values[test_ind]
    # fit model
    model = lgb.LGBMClassifier()
    model.fit(train[:,:-1], train[:,-1], eval_set=[(train[:,:-1], train[:,-1]), (X_valid, y_valid)], **lgb_fit_params)
    # evaluate model
    predictions = model.predict(test[:,:-1])
    test = resample(values, n_samples=n_size)
    predictions = model.predict(test[:,:-1])
    score = accuracy_score(test[:,-1], predictions)
    stats.append(score)
  alpha = 0.97
  p = ((1.0-alpha)/2.0) * 100
  lower = max(0.0, np.percentile(stats, p))
  p = (alpha+((1.0-alpha)/2.0)) * 100
  upper = min(1.0, np.percentile(stats, p))
  main = np.mean(stats)
  #print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))
  return main, np.max([main-lower, upper-main])

In [ ]:
def feature_acc(model, str_m, Rows):
  print(str_m)

  feature_imp=pd.DataFrame((zip(model.feature_importances_, photo_columns+agr_feature)), columns=['Model','Feature'])
  t = 3
  feature = feature_imp.sort_values(by='Model', ascending=False).iloc[:t]['Feature'].values
  X = df[feature].values

  data_split = data_preparation(X, y, Rows, 0.8)
  count = Rows//3

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]
  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)

  X_train, y_train = train[:,:-1], train[:,-1].astype('int')
  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_valid   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  y_valid   = np.concatenate((y1_test[count : 2*count], y2_test[count : 2*count], y3_test[count : 2*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)

  acc, err = bootstrap(X_test_norm, y_test, X_valid_norm, y_valid)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc_feature '+str_m+': '+str(acc)+'+-'+str(err)+', ')

In [ ]:
def ones(number_exp, Rows, Nd,	Na,	B,	BV,	mB,	λsparse,	Nsteps,	γ, learning_rate,	decay_rate,	decay_iterations,	shared, decision, mask_type):
  
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('number: '+str(number_exp)+', ')
  
  #data
  data_split = data_preparation(X, y, c=Rows//3)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = Rows//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)


  #classifier
  tn = TabNetClassifier( n_d=Nd, n_a=Na,
                          n_shared=shared, n_independent=decision,
                          momentum=mB,
                          optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=learning_rate),
                          scheduler_params={"step_size":decay_iterations, # how to use learning rate scheduler
                                            "gamma":decay_rate},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,

                          mask_type=mask_type,
                          **{ 'gamma': γ,
                              'lambda_sparse':λsparse,
                              'n_steps': Nsteps}

  )

  max_epochs = 2000

  patience=50
  t = time()
  tn.fit(
      X_train=X_train, y_train=y_train,
      #X_valid=X_valid, y_valid=y_valid,
      eval_set=[(X_train, y_train), (X_test, y_test)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss','accuracy'],
      max_epochs=max_epochs , patience=patience,
      batch_size=B, virtual_batch_size=BV,
  ) 
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time TN: '+str(t)+', ')

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 1000,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_valid_norm, y_valid)],  **lgb_fit_params)
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time GB: '+str(t)+', ')

  #Accuracy
  acc, err = bootstrap_accuracy(tn, X_test, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc TN: '+str(acc)+'+-'+str(err)+', ')
  acc, err = bootstrap_accuracy(gb, X_test_norm, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc GB: '+str(acc)+'+-'+str(err)+', ')

  #Feature importance
  feature_acc(tn, 'TN', 9000)
  feature_acc(gb, 'GB', 9000)

  #save model

  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('\n')
  gb.booster_.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/gb'+str(number_exp)+'.txt')
  tn.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/tn'+str(number_exp))

In [ ]:
ones(number_exp=0, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=1, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=2, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=3, 
     Rows=30000, 
     Nd=32,	Na=32,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=4, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=5, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=6, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.2, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=7, 
     Rows=30000, 
     Nd=128,	Na=128,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=8, 
     Rows=30000, 
     Nd=128,	Na=128,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=9, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=7,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=10, 
     Rows=9000, 
     Nd=64,	Na=64,	
     B=512,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
  number_exp=11
  Rows=30000
  Nd=64
  Na=64	
  B=512
  BV=128
  mB=0.7	
  λsparse=0.001
  Nsteps=5
  γ=1.5 
  learning_rate=0.02
  decay_rate=0.95
  decay_iterations=200	
  shared=2
  decision=2
  mask_type='entmax'

  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('number: '+str(11)+', ')
  
  #data
  data_split = data_preparation(X, y, c=Rows//3)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = Rows//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)


  #classifier
  tn = TabNetClassifier( n_d=Nd, n_a=Na,
                          n_shared=shared, n_independent=decision,
                          momentum=mB,
                          optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=learning_rate),
                          scheduler_params={"step_size":decay_iterations, # how to use learning rate scheduler
                                            "gamma":decay_rate},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,

                          mask_type=mask_type,
                          **{ 'gamma': γ,
                              'lambda_sparse':λsparse,
                              'n_steps': Nsteps}

  )

  max_epochs = 2000
  t = time()
  tn.fit(
      X_train=X_train, y_train=y_train,
      #X_valid=X_valid, y_valid=y_valid,
      eval_set=[(X_train, y_train), (X_test, y_test)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss','accuracy'],
      max_epochs=max_epochs , patience=20,
      batch_size=B, virtual_batch_size=BV,
  ) 
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time TN: '+str(t)+', ')

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'colsample_bytree': 0.6437405148446416,
    'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 100,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_valid_norm, y_valid)],  **lgb_fit_params)
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time GB: '+str(t)+', ')

  #Accuracy
  acc, err = bootstrap_accuracy(tn, X_test, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc TN: '+str(acc)+'+-'+str(err)+', ')
  acc, err = bootstrap_accuracy(gb, X_test_norm, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc GB: '+str(acc)+'+-'+str(err)+', ')

  #Feature importance
  feature_acc(tn, 'TN', Rows)
  feature_acc(gb, 'GB', Rows)

  #save model

  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('\n')
  gb.booster_.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/gb'+str(number_exp)+'.txt')
  tn.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/tn'+str(number_exp))

In [ ]:
ones(number_exp=12, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='sparsemax')

In [ ]:
ones(number_exp=13, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.2, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=14, 
     Rows=300000, 
     Nd=16,	Na=16,	
     B=16384,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.2, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.64086 | train_logloss: 8.10103 | train_accuracy: 0.32102 | valid_logloss: 8.10236 | valid_accuracy: 0.32096 |  0:00:11s
epoch 1  | loss: 0.36108 | train_logloss: 2.46956 | train_accuracy: 0.50289 | valid_logloss: 2.47166 | valid_accuracy: 0.50304 |  0:00:23s
epoch 2  | loss: 0.33227 | train_logloss: 6.48805 | train_accuracy: 0.18171 | valid_logloss: 6.4795  | valid_accuracy: 0.18392 |  0:00:34s
epoch 3  | loss: 0.3219  | train_logloss: 2.64671 | train_accuracy: 0.35143 | valid_logloss: 2.64231 | valid_accuracy: 0.35206 |  0:00:46s
epoch 4  | loss: 0.31253 | train_logloss: 2.006   | train_accuracy: 0.37547 | valid_logloss: 2.00072 | valid_accuracy: 0.37642 |  0:00:58s
epoch 5  | loss: 0.30635 | train_logloss: 2.46675 | train_accuracy: 0.22537 | valid_logloss: 2.46554 | valid_accuracy: 0.22494 |  0:01:10s
epoch 6  | loss: 0.30025 | train_logloss: 1.67759 | train_accuracy: 0.33518 | valid_logloss: 1.67998 | valid_accuracy: 0.335   |  0:01:21s
epoch 7 

UnboundLocalError: ignored

In [ ]:
ones(number_exp=15, 
     Rows=300000, 
     Nd=16,	Na=16,	
     B=16384,	BV=1024,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=14, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=16, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.07899 | train_logloss: 16.10763| train_accuracy: 0.36503 | valid_logloss: 16.06683| valid_accuracy: 0.3691  |  0:00:02s
epoch 1  | loss: 0.61676 | train_logloss: 6.73377 | train_accuracy: 0.46913 | valid_logloss: 6.67126 | valid_accuracy: 0.4731  |  0:00:04s
epoch 2  | loss: 0.51035 | train_logloss: 3.21475 | train_accuracy: 0.3945  | valid_logloss: 3.27889 | valid_accuracy: 0.39143 |  0:00:06s
epoch 3  | loss: 0.47394 | train_logloss: 3.77001 | train_accuracy: 0.382   | valid_logloss: 3.71168 | valid_accuracy: 0.38347 |  0:00:08s
epoch 4  | loss: 0.41743 | train_logloss: 2.41424 | train_accuracy: 0.36523 | valid_logloss: 2.4196  | valid_accuracy: 0.36487 |  0:00:10s
epoch 5  | loss: 0.41772 | train_logloss: 1.25983 | train_accuracy: 0.52977 | valid_logloss: 1.25954 | valid_accuracy: 0.52567 |  0:00:12s
epoch 6  | loss: 0.38943 | train_logloss: 1.25853 | train_accuracy: 0.48813 | valid_logloss: 1.26784 | valid_accuracy: 0.48097 |  0:00:15s
epoch 7 

In [ ]:
ones(number_exp=17, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.9,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.07515 | train_logloss: 15.61666| train_accuracy: 0.40127 | valid_logloss: 15.62071| valid_accuracy: 0.4015  |  0:00:02s
epoch 1  | loss: 0.61204 | train_logloss: 22.99017| train_accuracy: 0.33367 | valid_logloss: 22.98541| valid_accuracy: 0.3337  |  0:00:04s
epoch 2  | loss: 0.52328 | train_logloss: 9.73707 | train_accuracy: 0.29887 | valid_logloss: 9.83167 | valid_accuracy: 0.29647 |  0:00:06s
epoch 3  | loss: 0.47488 | train_logloss: 10.96129| train_accuracy: 0.2991  | valid_logloss: 11.01346| valid_accuracy: 0.30263 |  0:00:08s
epoch 4  | loss: 0.4553  | train_logloss: 6.16226 | train_accuracy: 0.51473 | valid_logloss: 6.23659 | valid_accuracy: 0.5146  |  0:00:10s
epoch 5  | loss: 0.46163 | train_logloss: 4.28868 | train_accuracy: 0.4412  | valid_logloss: 4.31423 | valid_accuracy: 0.44427 |  0:00:12s
epoch 6  | loss: 0.43387 | train_logloss: 5.13905 | train_accuracy: 0.3385  | valid_logloss: 5.17095 | valid_accuracy: 0.3398  |  0:00:14s
epoch 7 

In [ ]:
ones(number_exp=18, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.0001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.0721  | train_logloss: 20.48473| train_accuracy: 0.3116  | valid_logloss: 20.52434| valid_accuracy: 0.31487 |  0:00:02s
epoch 1  | loss: 0.62702 | train_logloss: 7.26988 | train_accuracy: 0.28423 | valid_logloss: 7.35665 | valid_accuracy: 0.28297 |  0:00:04s
epoch 2  | loss: 0.4926  | train_logloss: 4.78337 | train_accuracy: 0.3647  | valid_logloss: 4.88082 | valid_accuracy: 0.36347 |  0:00:06s
epoch 3  | loss: 0.44313 | train_logloss: 3.25561 | train_accuracy: 0.29043 | valid_logloss: 3.28029 | valid_accuracy: 0.29257 |  0:00:08s
epoch 4  | loss: 0.4382  | train_logloss: 5.81552 | train_accuracy: 0.35523 | valid_logloss: 5.85558 | valid_accuracy: 0.35263 |  0:00:10s
epoch 5  | loss: 0.42124 | train_logloss: 3.80157 | train_accuracy: 0.3198  | valid_logloss: 3.82596 | valid_accuracy: 0.32247 |  0:00:12s
epoch 6  | loss: 0.40068 | train_logloss: 2.142   | train_accuracy: 0.3114  | valid_logloss: 2.15032 | valid_accuracy: 0.31263 |  0:00:14s
epoch 7 

In [ ]:
  c1 = 9000
  data_split = data_preparation(X, y, c=c1//3)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = c1//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'colsample_bytree': 0.6437405148446416,
    'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 10000,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_valid_norm, y_valid)],  **lgb_fit_params)
  t = time()-t
  print(t)

  data_split = data_preparation(X, y, c=30000//3)


  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = 30000//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)


  #Accuracy
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('GB '+str(c1)+'   ')

  acc, err = bootstrap_accuracy(gb, X_test_norm, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc GB: '+str(acc)+'+-'+str(err)+', ')

  #Feature importance
  feature_acc(gb, 'GB', Rows)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('\n')
  gb.booster_.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/gb_size'+str(c1)+'.txt')